In [2]:
import cv2

In [3]:
img =cv2.imread('bulbasaur.png')


In [4]:
cv2.imshow('bulbasaur image',img)

In [5]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    cv2.imshow("video frame",frame)
    
    #wait fro user input -q key for quit
    
    key_pressed=cv2.waitKey(1)&0xFF
    if key_pressed==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# haarcscade_frontalface classifier

In [8]:
import numpy as np
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path='data/'

file_name=input("enter the name of the person")
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
        
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    
    
    
    #PICK THE LAST FACE (BECAUSE IT IS THE FACE ACC TO AREA)
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        
        #extract (crop out the required face): region of interest
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        skip+=1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))
            
        
    cv2.imshow("video  frame",frame)
    cv2.imshow("face section",face_section)
    #wait fro user input -q key for quit
    
    key_pressed=cv2.waitKey(1)&0xFF
    if key_pressed==ord('q'):
        break
        
        
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
cap.release()
print(face_data.shape)
np.save(dataset_path+file_name+'.npy',face_data)
print("data save successfully")

cv2.destroyAllWindows()

enter the name of the personlakshay
1
2
3
4
5
6
7
8
9
10
11
12
13
14
(14, 30000)
data save successfully


In [9]:
import cv2
import numpy as np
import os

def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(x,y,query,k=5):
    
    vals=[]
    n=x.shape[0] 
    
    for i in range(n):
        d=dist(query,x[i])
        vals.append((d,y[i]))
    vals=sorted(vals)    
    vals=vals[:k]
    vals=np.array(vals)
    
    new_val=np.unique(vals[:,1],return_counts=True)
    index=new_val[1].argmax()
    pred=new_val[0][index]"haarcascade_frontalface_alt.xml"
    
    return pred

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier()
skip=0
face_data=[]
dataset_path='data/'
label=[]
class_id=0 #labels for then given file 
names={} #mapping btw id-name

#data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(label,axis=0)

while True:
    ret,frame=cap.read()
    if ret==False:
        continue
        
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h=face
        
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+h+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        out=knn(face_dataset,face_labels,face_section.flatten())
        
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("faces",frame)
    
    
    key_pressed=cv2.waitKey(1)&0xFF
    if key_pressed==ord('q'):
        break
cap.release()

cv2.destroyAllWindows()
    



error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-t9hleyt8\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
